In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'
import json


# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient

# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

# Load Premotor table

In [ ]:
pre_to_df = connectome_create.load_pre_to_df(ext='ordered_no_frags')
pre_to_df.loc['motor'].index.to_frame().set_index('preferred_pool').to_csv('./dfs_saved/mn_to_pool_match.csv')

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()
def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(np.log10(df.T.to_numpy()+1), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if savefig:
        if figfilename is None:
            figfilename = 'flexor_pool_temp_' + (np.random.randint(10000,99999,size=None)).astype(str)
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def lin_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(df.T.to_numpy(), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'linear', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if figfilename is not None:
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def keep_first_string_in_list(in_list):
    out_list = []
    key = None
    for s in in_list:
        if key is None:
            key = s
            out_list = out_list + [key]
            continue
        elif s==key:
            out_list = out_list + ['']
        else:
            key = s
            out_list = out_list + [s]
    return out_list

# Now look at the small slice of local premotor neurons that prefer flexors, onto everything

In [ ]:
All = slice(None)
pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor_med_venU',
        'tarsus_depressor_noid',
        ]

# input: ['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'segID']
# output: ['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'post_pt_root_id']

In [ ]:
# first sort by synapses onto the pool
input_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
pre_onto_ti_ext_flex_df = pre_to_df.loc[:,input_tup]

## Show the Motor neurons first

In [ ]:
pool_list = ['tibia_extensor','main_tibia_flexor', 'auxiliary_tibia_flexor_B','auxiliary_tibia_flexor_E']
input_tup = ('local',pool_list,All,All,All,All)
output_tup = ('motor',pool_list,All,All,All,All,All)

pre_onto_ti_mns_df = pre_to_df.loc[output_tup,input_tup]
# weird neurons
print('no NT: {}'.format(pre_onto_ti_mns_df.loc[:,~pre_onto_ti_mns_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_mns_df = pre_onto_ti_mns_df.loc[:,pre_onto_ti_mns_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# xticklabels = pre_onto_ti_ext_flex_df.index.get_level_values('preferred_pool')
# yticklabels = pre_df_ti_ext_flex.columns.get_level_values('NT')

# conn_heat_map(df=pre_df_ti_ext_flex, xticks=xticklabels,yticks=yticklabels,figsz=[4,16])

# order by NT
cols_ntsorted,c_array = pre_onto_ti_mns_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
mn_nt_sorted = pre_onto_ti_mns_df.loc[:,cols_ntsorted]

# order by pool
cols_ntsorted,c_array = mn_nt_sorted.columns.sortlevel(level='preferred_pool',ascending=True,sort_remaining=False)
output_tup = ('local',pool_list,All,All,All,All,All)
mn_nt_pool_sorted = mn_nt_sorted.loc[:,output_tup]

mnpoollabel = mn_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = mn_nt_pool_sorted.columns.get_level_values('NT')

conn_heat_map(df=mn_nt_pool_sorted, xticks=mnpoollabel,yticks=yticklabels,figsz=[4,16])

## Onto local neurons preferring femur pools

In [ ]:
pool_list = ['tibia_extensor','main_tibia_flexor', 'auxiliary_tibia_flexor_B','auxiliary_tibia_flexor_E']
input_tup = ('local',pool_list,All,All,All,All)
output_tup = ('local',pool_list,All,All,All,All,All)

pre_onto_ti_local_df = pre_to_df.loc[output_tup,input_tup]


# weird neurons
print('no NT: {}'.format(pre_onto_ti_local_df.loc[:,~pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_local_df = pre_onto_ti_local_df.loc[pre_onto_ti_local_df.index.get_level_values('NT').isin(['Ach','GABA','Glu']),pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# sort by 
rows_ntsorted,array = pre_onto_ti_local_df.index.sortlevel(level='NT',ascending=True,sort_remaining=False)
cols_ntsorted,c_array = pre_onto_ti_local_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
local_nt_sorted = pre_onto_ti_local_df.loc[rows_ntsorted,cols_ntsorted]

# order by pool
local_nt_pool_sorted = local_nt_sorted.loc[output_tup,input_tup]

# mnpoollabel = local_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = local_nt_pool_sorted.columns.get_level_values('NT')

xpool = local_nt_pool_sorted.index.get_level_values('preferred_pool')
# pool_dict = {'tibia_extensor':'ext','main_tibia_flexor':'flex'}
# xpool = [pool_dict[key] for key in xpool]

xNT = local_nt_pool_sorted.index.get_level_values('NT').to_list()
xpoolnt = [x + "_" + y for x,y in zip(xpool,xNT)]

yticklabels = keep_first_string_in_list(local_nt_pool_sorted.columns.get_level_values('preferred_pool').to_list())
xpoolnt = keep_first_string_in_list(xpoolnt)

conn_heat_map(df=local_nt_pool_sorted, xticks=xpoolnt,yticks=xpoolnt,figsz=[16,16])

# Flexor neurons onto other pools

In [ ]:
pool_list = ['tibia_extensor','main_tibia_flexor', 'auxiliary_tibia_flexor_B','auxiliary_tibia_flexor_E']
input_tup = ('local',pool_list,All,All,All,All)
output_tup = ('local',pool_list,All,All,All,All,All)

pre_onto_ti_mns_df = pre_to_df.loc[output_tup,input_tup]
# weird neurons
print('no NT: {}'.format(pre_onto_ti_mns_df.loc[:,~pre_onto_ti_mns_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_mns_df = pre_onto_ti_mns_df.loc[:,pre_onto_ti_mns_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# xticklabels = pre_onto_ti_ext_flex_df.index.get_level_values('preferred_pool')
# yticklabels = pre_df_ti_ext_flex.columns.get_level_values('NT')

# conn_heat_map(df=pre_df_ti_ext_flex, xticks=xticklabels,yticks=yticklabels,figsz=[4,16])

# order by NT
cols_ntsorted,c_array = pre_onto_ti_mns_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
mn_nt_sorted = pre_onto_ti_mns_df.loc[:,cols_ntsorted]

# order by pool
cols_ntsorted,c_array = mn_nt_sorted.columns.sortlevel(level='preferred_pool',ascending=True,sort_remaining=False)
output_tup = ('local',pool_list,All,All,All,All,All)
mn_nt_pool_sorted = mn_nt_sorted.loc[output_tup,output_tup]

mnpoollabel = mn_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = mn_nt_pool_sorted.columns.get_level_values('NT')

conn_heat_map(df=mn_nt_pool_sorted, xticks=mnpoollabel,yticks=yticklabels,figsz=[24,24])

## Onto all other local neurons in all pools

In [ ]:
input_tup = ('local',['tibia_extensor','main_tibia_flexor', 'auxiliary_tibia_flexor_B','auxiliary_tibia_flexor_E'],All,All,All,All)
output_tup = ('local',All,All,All,All,All,All)

pre_onto_ti_local_df = pre_to_df.loc[output_tup,input_tup]


# weird neurons
print('no NT: {}'.format(pre_onto_ti_local_df.loc[:,~pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_local_df = pre_onto_ti_local_df.loc[pre_onto_ti_local_df.index.get_level_values('NT').isin(['Ach','GABA','Glu']),pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# sort by 
rows_ntsorted,array = pre_onto_ti_local_df.index.sortlevel(level='NT',ascending=True,sort_remaining=False)
cols_ntsorted,c_array = pre_onto_ti_local_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
local_nt_sorted = pre_onto_ti_local_df.loc[rows_ntsorted,cols_ntsorted]

# order by pool
output_tup = ('local',pool_keys,All,All,All,All,All)
local_nt_pool_sorted = local_nt_sorted.loc[output_tup,input_tup]

# mnpoollabel = local_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = local_nt_pool_sorted.columns.get_level_values('NT')

xpool = local_nt_pool_sorted.index.get_level_values('preferred_pool')
# pool_dict = {'tibia_extensor':'ext','main_tibia_flexor':'flex'}
# xpool = [pool_dict[key] for key in xpool]

xNT = local_nt_pool_sorted.index.get_level_values('NT').to_list()
xpoolnt = [x + "_" + y for x,y in zip(xpool,xNT)]

yticklabels = keep_first_string_in_list(local_nt_pool_sorted.columns.get_level_values('preferred_pool').to_list())
xpoolnt = keep_first_string_in_list(xpoolnt)

conn_heat_map(df=local_nt_pool_sorted, xticks=xpoolnt,yticks=yticklabels,figsz=[24,8])

## Finally, all local onto all other local neurons in all pools

In [ ]:
input_tup = ('local',pool_keys,All,All,All,All)
output_tup = ('local',pool_keys,All,All,All,All,All)

pre_onto_ti_local_df = pre_to_df.loc[output_tup,input_tup]


# weird neurons
print('no NT: {}'.format(pre_onto_ti_local_df.loc[:,~pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_local_df = pre_onto_ti_local_df.loc[pre_onto_ti_local_df.index.get_level_values('NT').isin(['Ach','GABA','Glu']),pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# sort by 
rows_ntsorted,array = pre_onto_ti_local_df.index.sortlevel(level='NT',ascending=True,sort_remaining=False)
cols_ntsorted,c_array = pre_onto_ti_local_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
local_nt_sorted = pre_onto_ti_local_df.loc[rows_ntsorted,cols_ntsorted]

# order by pool
output_tup = ('local',pool_keys,All,All,All,All,All)
local_nt_pool_sorted = local_nt_sorted.loc[output_tup,input_tup]

# mnpoollabel = local_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = local_nt_pool_sorted.columns.get_level_values('NT')

xpool = local_nt_pool_sorted.index.get_level_values('preferred_pool')
# pool_dict = {'tibia_extensor':'ext','main_tibia_flexor':'flex'}
# xpool = [pool_dict[key] for key in xpool]

xNT = local_nt_pool_sorted.index.get_level_values('NT').to_list()
xpoolnt = [x + "_" + y for x,y in zip(xpool,xNT)]

yticklabels = keep_first_string_in_list(local_nt_pool_sorted.columns.get_level_values('preferred_pool').to_list())
xpoolnt = keep_first_string_in_list(xpoolnt)

conn_heat_map(df=local_nt_pool_sorted, xticks=xpoolnt,yticks=yticklabels,figsz=[24,24])

In [ ]:
xpool

In [ ]:

# weird neurons
print('no NT: {}'.format(pre_onto_ti_local_df.loc[:,~pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])].columns.get_level_values('segID').to_list()))
# Get rid of the last two pMNs
pre_onto_ti_local_df = pre_onto_ti_local_df.loc[pre_onto_ti_local_df.index.get_level_values('NT').isin(['Ach','GABA','Glu']),pre_onto_ti_local_df.columns.get_level_values('NT').isin(['Ach','GABA','Glu'])]

# sort by 
rows_ntsorted,array = pre_onto_ti_local_df.index.sortlevel(level='NT',ascending=True,sort_remaining=False)
cols_ntsorted,c_array = pre_onto_ti_local_df.columns.sortlevel(level='NT',ascending=True,sort_remaining=False)
local_nt_sorted = pre_onto_ti_local_df.loc[rows_ntsorted,cols_ntsorted]

# order by pool
output_tup = ('local',['tibia_extensor','main_tibia_flexor'],All,All,All,All,All)
local_nt_pool_sorted = local_nt_sorted.loc[output_tup,output_tup]

# mnpoollabel = local_nt_pool_sorted.index.get_level_values('preferred_pool')
yticklabels = local_nt_pool_sorted.columns.get_level_values('NT')

xpool = local_nt_pool_sorted.index.get_level_values('preferred_pool')
pool_dict = {'tibia_extensor':'ext','main_tibia_flexor':'flex'}
xpool = [pool_dict[key] for key in xpool]

xNT = local_nt_pool_sorted.index.get_level_values('NT').to_list()
xpoolnt = [x + "_" + y for x,y in zip(xpool,xNT)]

conn_heat_map(df=local_nt_pool_sorted, xticks=xpoolnt,yticks=xpoolnt,figsz=[16,16])

In [ ]:
# Format a nicer figure with both plots side by side
# do it in different colors too, then cut out different versions in illustrator
fig, ax = plt.subplots(1, 2, figsize=(32,16),gridspec_kw={'width_ratios': [1, 3.5]})

cmap = utils.white_dense()
# cmap = sns.color_palette("light:seagreen", as_cmap=True)

# def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
sns.heatmap(np.log10(mn_nt_pool_sorted.T.to_numpy()+1),ax=ax[0], xticklabels=mnpoollabel, yticklabels=xpoolnt, cmap=cmap,square=True,cbar=False,vmax=np.log10(633),vmin=0)
plt.xlabel('Onto MNs', fontsize =18)
plt.ylabel('From input neurons', fontsize =18)
sns.heatmap(np.log10(local_nt_pool_sorted.T.to_numpy()+1),ax=ax[1], xticklabels=xpoolnt, yticklabels=xpoolnt,square=True, cmap=cmap,vmax=np.log10(633),vmin=0)
plt.xlabel('Onto local pmns', fontsize =18)

cbar = ax[1].collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses', size=24)
# plt.yticks(fontsize = 16)
# plt.xticks(fontsize = 16)
# plt.show()

fig.savefig('./figpanels/' + 'local_ti_ext_flex_by_nt' + '.svg',format='svg')


# Pick out some interesting neurons
Plot the inputs and output, synapses?

In [ ]:
def build_synapses(sid,syn_df):

    # variables to render in neuroglancer link
    render_synapses = syn_df
    render_neurons = [sid]

    img_source = client.info.image_source()
    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=img_source,
                                 )
    seg_source = client.info.segmentation_source()

    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source = seg_source,
                                        fixed_ids = render_neurons)
    points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
    anno_layer = ngstbld.AnnotationLayerConfig(name='annos',
                                       mapping_rules=points )

    sb = ngstbld.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
    return sb.render_state(render_synapses, return_as='json')

In [ ]:
# 5) Convergent input from Glu neurons onto big 21A neuronn
pre_mn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','Glu',All,All),('local','main_tibia_flexor','Glu',All,All)].iloc[[0],:]
onto_list = pre_mn.index.get_level_values('post_pt_root_id').to_list()
from_list = pre_mn.columns.get_level_values('segID').to_list()

timestamp = connectome_create.get_timestamp()
iln_syn_df = client.materialize.synapse_query(post_ids = onto_list, pre_ids=from_list, timestamp=timestamp) # Takes list

%matplotlib inline
ngjson = build_synapses(onto_list[0],iln_syn_df)

with open('./syn_jsons/glu_ng_json.json', 'w', encoding='utf-8') as f:
    json.dump(ngjson, f, ensure_ascii=False, indent=4)

# GABA
pre_mn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','Glu',All,All),('local','main_tibia_flexor','GABA',All,All)].iloc[[0],:]
onto_list = pre_mn.index.get_level_values('post_pt_root_id').to_list()
from_list = pre_mn.columns.get_level_values('segID').to_list()

timestamp = connectome_create.get_timestamp()
iln_syn_df = client.materialize.synapse_query(post_ids = onto_list, pre_ids=from_list, timestamp=timestamp) # Takes list

%matplotlib inline
ngjson = build_synapses(onto_list[0],iln_syn_df)
ngjson

# Ach
pre_mn = local_nt_pool_sorted.loc[('local','main_tibia_flexor','Glu',All,All),('local','main_tibia_flexor','Ach',All,All)].iloc[[0],:]
onto_list = pre_mn.index.get_level_values('post_pt_root_id').to_list()
from_list = pre_mn.columns.get_level_values('segID').to_list()

timestamp = connectome_create.get_timestamp()
iln_syn_df = client.materialize.synapse_query(post_ids = onto_list, pre_ids=from_list, timestamp=timestamp) # Takes list

ngjson = build_synapses(onto_list[0],iln_syn_df)
ngjson
